# Evolução de Schema com Apache Iceberg

Este notebook demonstra uma das funcionalidades mais poderosas do Apache Iceberg: a capacidade de evoluir o schema de tabelas sem interrupção de serviço ou reescrita de dados.

## Contexto e Objetivos:

### Desafios Tradicionais com Schema:
- **Formatos Tradicionais**: Parquet, ORC requerem reescrita completa para mudanças de schema
- **Downtime**: Necessidade de parar aplicações durante alterações
- **Complexidade**: Processos manuais e propensos a erro
- **Custo**: Reprocessamento de grandes volumes de dados

### Benefícios da Schema Evolution no Iceberg:
- **Zero Downtime**: Alterações sem interrupção de serviço
- **Compatibilidade**: Dados antigos permanecem acessíveis
- **Flexibilidade**: Adição, remoção e renomeação de colunas
- **Versionamento**: Histórico completo de mudanças de schema
- **Performance**: Sem necessidade de reescrita de dados

### Tipos de Evolução Suportados:
1. **ADD COLUMN**: Adicionar novas colunas
2. **DROP COLUMN**: Remover colunas existentes
3. **RENAME COLUMN**: Renomear colunas
4. **ALTER COLUMN TYPE**: Mudanças compatíveis de tipo
5. **REORDER COLUMNS**: Reorganizar ordem das colunas

## Setup do Ambiente

In [4]:
# Para o Spark se estiver rodando
try:
    spark.stop()
except:
    pass

# Configuração do Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark-3.3.0-bin-hadoop3"

import findspark
findspark.init('/opt/spark-3.3.0-bin-hadoop3')

from pyspark.sql import SparkSession

# Sessão Spark
spark = SparkSession.builder \
    .appName("IcebergSchemaEvolution") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.hadoop_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.hadoop_catalog.type", "hadoop") \
    .config("spark.sql.catalog.hadoop_catalog.warehouse", "file:///home/tavares/warehouse") \
    .config("spark.sql.default.catalog", "hadoop_catalog") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .getOrCreate()

print(f"Warehouse configurado para: {spark.conf.get('spark.sql.catalog.hadoop_catalog.warehouse')}")

Warehouse configurado para: file:///home/tavares/warehouse


## 1. Criação da Tabela Inicial

Vamos começar criando uma tabela com um schema básico que será evoluído ao longo do exemplo.

In [5]:
# Exclui a tabela se existir
spark.sql("DROP TABLE IF EXISTS hadoop_catalog.default.vendas")

# Cria a tabela Vendas no catálogo, usando Iceberg
spark.sql("""
    CREATE TABLE hadoop_catalog.default.vendas (
        id INT,
        produto STRING,
        quantidade INT,
        preco DOUBLE,
        data_venda DATE
    )
    USING iceberg
""")

DataFrame[]

## 2. Inserção de Dados Iniciais

Vamos inserir alguns dados na tabela com o schema original.

In [6]:
# Incluindo dados na tabela vendas usando SQL puro
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas VALUES
    (1, 'Produto A', 10, 15.5, DATE('2024-11-01')),
    (2, 'Produto B', 5, 22.0, DATE('2024-11-02')),
    (3, 'Produto C', 8, 30.0, DATE('2024-11-03'))
""")

DataFrame[]

## 3. Verificação dos Dados Iniciais

Vamos verificar os dados inseridos e o schema atual da tabela.

In [7]:
# Verificamos os dados
print("=== Dados Iniciais ===")
spark.sql("SELECT * FROM hadoop_catalog.default.vendas").show()

# Verificamos o schema atual
print("\n=== Schema Inicial ===")
spark.sql("DESCRIBE hadoop_catalog.default.vendas").show()

=== Dados Iniciais ===
+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2024-11-01|
|  2|Produto B|         5| 22.0|2024-11-02|
|  3|Produto C|         8| 30.0|2024-11-03|
+---+---------+----------+-----+----------+


=== Schema Inicial ===
+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|             id|      int|       |
|        produto|   string|       |
|     quantidade|      int|       |
|          preco|   double|       |
|     data_venda|     date|       |
|               |         |       |
| # Partitioning|         |       |
|Not partitioned|         |       |
+---------------+---------+-------+



## 4. Evolução do Schema - Adicionando Coluna

Agora vamos demonstrar a primeira evolução: adicionar uma nova coluna `desconto` à tabela existente. Esta operação é instantânea e não requer reescrita dos dados existentes.

In [8]:
# Adicionando nova coluna desconto
spark.sql("""
    ALTER TABLE hadoop_catalog.default.vendas
    ADD COLUMN desconto DOUBLE
""")

print("Coluna 'desconto' adicionada com sucesso!")

Coluna 'desconto' adicionada com sucesso!


## 5. Verificação do Schema Evoluído

Vamos verificar como o schema foi alterado e como os dados existentes são tratados.

In [9]:
# Schema Atualizado
print("=== Schema Após Adição de Coluna ===")
spark.sql("DESCRIBE hadoop_catalog.default.vendas").show()

# Verificar dados existentes (nova coluna será NULL)
print("\n=== Dados Após Evolução (note os valores NULL) ===")
spark.sql("SELECT * FROM hadoop_catalog.default.vendas").show()

=== Schema Após Adição de Coluna ===
+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|             id|      int|       |
|        produto|   string|       |
|     quantidade|      int|       |
|          preco|   double|       |
|     data_venda|     date|       |
|       desconto|   double|       |
|               |         |       |
| # Partitioning|         |       |
|Not partitioned|         |       |
+---------------+---------+-------+


=== Dados Após Evolução (note os valores NULL) ===
+---+---------+----------+-----+----------+--------+
| id|  produto|quantidade|preco|data_venda|desconto|
+---+---------+----------+-----+----------+--------+
|  1|Produto A|        10| 15.5|2024-11-01|    null|
|  2|Produto B|         5| 22.0|2024-11-02|    null|
|  3|Produto C|         8| 30.0|2024-11-03|    null|
+---+---------+----------+-----+----------+--------+



## 6. Inserção de Dados com Nova Coluna

Agora vamos inserir novos dados que incluem valores para a coluna `desconto` recém-adicionada.

In [10]:
# Inserindo dados com a nova coluna
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas VALUES
    (4, 'Produto D', 12, 25.0, DATE('2024-11-04'), 2.5),
    (5, 'Produto E', 7, 18.0, DATE('2024-11-05'), 1.0)
""")

print("Novos dados inseridos com valores de desconto!")

Novos dados inseridos com valores de desconto!


## 7. Verificação Final dos Dados

Vamos ver como a tabela agora contém dados com e sem a nova coluna, demonstrando a compatibilidade retroativa.

In [11]:
# Consultando todos os dados
print("=== Dados Finais (Mistura de Registros Antigos e Novos) ===")
spark.sql("SELECT * FROM hadoop_catalog.default.vendas ORDER BY id").show()

# Estatísticas da tabela
print("\n=== Estatísticas ===")
spark.sql("SELECT COUNT(*) as total_registros FROM hadoop_catalog.default.vendas").show()
spark.sql("SELECT COUNT(*) as com_desconto FROM hadoop_catalog.default.vendas WHERE desconto IS NOT NULL").show()
spark.sql("SELECT COUNT(*) as sem_desconto FROM hadoop_catalog.default.vendas WHERE desconto IS NULL").show()

=== Dados Finais (Mistura de Registros Antigos e Novos) ===
+---+---------+----------+-----+----------+--------+
| id|  produto|quantidade|preco|data_venda|desconto|
+---+---------+----------+-----+----------+--------+
|  1|Produto A|        10| 15.5|2024-11-01|    null|
|  2|Produto B|         5| 22.0|2024-11-02|    null|
|  3|Produto C|         8| 30.0|2024-11-03|    null|
|  4|Produto D|        12| 25.0|2024-11-04|     2.5|
|  5|Produto E|         7| 18.0|2024-11-05|     1.0|
+---+---------+----------+-----+----------+--------+


=== Estatísticas ===
+---------------+
|total_registros|
+---------------+
|              5|
+---------------+

+------------+
|com_desconto|
+------------+
|           2|
+------------+

+------------+
|sem_desconto|
+------------+
|           3|
+------------+



## 8. Mais Evoluções de Schema

Vamos demonstrar outras operações de evolução de schema disponíveis no Iceberg.

In [12]:
# Adicionando mais uma coluna
spark.sql("""
    ALTER TABLE hadoop_catalog.default.vendas
    ADD COLUMN categoria STRING
""")

# Renomeando uma coluna
spark.sql("""
    ALTER TABLE hadoop_catalog.default.vendas
    RENAME COLUMN produto TO nome_produto
""")

print("Evoluções adicionais aplicadas!")

# Verificar schema final
print("\n=== Schema Final ===")
spark.sql("DESCRIBE hadoop_catalog.default.vendas").show()

Evoluções adicionais aplicadas!

=== Schema Final ===
+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|             id|      int|       |
|   nome_produto|   string|       |
|     quantidade|      int|       |
|          preco|   double|       |
|     data_venda|     date|       |
|       desconto|   double|       |
|      categoria|   string|       |
|               |         |       |
| # Partitioning|         |       |
|Not partitioned|         |       |
+---------------+---------+-------+



## 9. Inserção com Schema Completo

Vamos inserir dados usando o schema completamente evoluído.

In [13]:
# Inserir dados com todas as colunas
spark.sql("""
    INSERT INTO hadoop_catalog.default.vendas VALUES
    (6, 'Produto F', 15, 35.0, DATE('2024-11-06'), 3.0, 'Eletrônicos'),
    (7, 'Produto G', 20, 45.0, DATE('2024-11-07'), 5.0, 'Casa')
""")

print("Dados inseridos com schema completo!")

# Visualizar resultado final
print("\n=== Dados Finais com Schema Evoluído ===")
spark.sql("SELECT * FROM hadoop_catalog.default.vendas ORDER BY id").show()

Dados inseridos com schema completo!

=== Dados Finais com Schema Evoluído ===
+---+------------+----------+-----+----------+--------+-----------+
| id|nome_produto|quantidade|preco|data_venda|desconto|  categoria|
+---+------------+----------+-----+----------+--------+-----------+
|  1|   Produto A|        10| 15.5|2024-11-01|    null|       null|
|  2|   Produto B|         5| 22.0|2024-11-02|    null|       null|
|  3|   Produto C|         8| 30.0|2024-11-03|    null|       null|
|  4|   Produto D|        12| 25.0|2024-11-04|     2.5|       null|
|  5|   Produto E|         7| 18.0|2024-11-05|     1.0|       null|
|  6|   Produto F|        15| 35.0|2024-11-06|     3.0|Eletrônicos|
|  7|   Produto G|        20| 45.0|2024-11-07|     5.0|       Casa|
+---+------------+----------+-----+----------+--------+-----------+



## 10. Histórico de Snapshots

Vamos verificar como cada evolução de schema criou novos snapshots, permitindo auditoria completa das mudanças.

In [14]:
# Verificar snapshots criados durante as evoluções
print("=== Histórico de Snapshots ===")
spark.sql("""
    SELECT snapshot_id, committed_at, operation
    FROM hadoop_catalog.default.vendas.snapshots 
    ORDER BY committed_at
""").show(truncate=False)

# Verificar histórico da tabela
print("\n=== Histórico da Tabela ===")
spark.sql("SELECT * FROM hadoop_catalog.default.vendas.history").show(truncate=False)

=== Histórico de Snapshots ===
+-------------------+-----------------------+---------+
|snapshot_id        |committed_at           |operation|
+-------------------+-----------------------+---------+
|2785368502997395094|2025-11-02 23:13:40.555|append   |
|4794735984909422061|2025-11-02 23:13:49.538|append   |
|2939522097528125544|2025-11-02 23:14:17.6  |append   |
+-------------------+-----------------------+---------+


=== Histórico da Tabela ===
+-----------------------+-------------------+-------------------+-------------------+
|made_current_at        |snapshot_id        |parent_id          |is_current_ancestor|
+-----------------------+-------------------+-------------------+-------------------+
|2025-11-02 23:13:40.555|2785368502997395094|null               |true               |
|2025-11-02 23:13:49.538|4794735984909422061|2785368502997395094|true               |
|2025-11-02 23:14:17.6  |2939522097528125544|4794735984909422061|true               |
+-----------------------+------

## 11. Time Travel com Schemas Diferentes

Demonstração de como consultar dados usando schemas de diferentes pontos no tempo.

In [15]:
# Consultar dados do primeiro snapshot (schema original)
snapshots = spark.sql("SELECT snapshot_id FROM hadoop_catalog.default.vendas.snapshots ORDER BY committed_at")
first_snapshot = snapshots.collect()[0][0]

print(f"Consultando primeiro snapshot: {first_snapshot}")
print("=== Dados com Schema Original ===")
spark.sql(f"""
    SELECT * FROM hadoop_catalog.default.vendas 
    VERSION AS OF {first_snapshot}
    ORDER BY id
""").show()

print("\n=== Dados com Schema Atual ===")
spark.sql("SELECT * FROM hadoop_catalog.default.vendas ORDER BY id").show()

Consultando primeiro snapshot: 2785368502997395094
=== Dados com Schema Original ===
+---+---------+----------+-----+----------+
| id|  produto|quantidade|preco|data_venda|
+---+---------+----------+-----+----------+
|  1|Produto A|        10| 15.5|2024-11-01|
|  2|Produto B|         5| 22.0|2024-11-02|
|  3|Produto C|         8| 30.0|2024-11-03|
+---+---------+----------+-----+----------+


=== Dados com Schema Atual ===
+---+------------+----------+-----+----------+--------+-----------+
| id|nome_produto|quantidade|preco|data_venda|desconto|  categoria|
+---+------------+----------+-----+----------+--------+-----------+
|  1|   Produto A|        10| 15.5|2024-11-01|    null|       null|
|  2|   Produto B|         5| 22.0|2024-11-02|    null|       null|
|  3|   Produto C|         8| 30.0|2024-11-03|    null|       null|
|  4|   Produto D|        12| 25.0|2024-11-04|     2.5|       null|
|  5|   Produto E|         7| 18.0|2024-11-05|     1.0|       null|
|  6|   Produto F|        15| 

## Resumo dos Benefícios da Schema Evolution

### Vantagens Demonstradas:

1. **Zero Downtime**: Todas as alterações foram aplicadas sem interrupção
2. **Compatibilidade Retroativa**: Dados antigos permanecem acessíveis
3. **Flexibilidade**: Múltiplos tipos de evolução suportados
4. **Versionamento**: Histórico completo de mudanças
5. **Time Travel**: Acesso a dados com schemas históricos

### Operações Suportadas:

- ✅ **ADD COLUMN**: Adicionar novas colunas
- ✅ **RENAME COLUMN**: Renomear colunas existentes
- ✅ **DROP COLUMN**: Remover colunas (não demonstrado)
- ✅ **ALTER COLUMN TYPE**: Mudanças compatíveis de tipo
- ✅ **REORDER COLUMNS**: Reorganizar ordem das colunas

### Casos de Uso Empresariais:

- **Evolução de Aplicações**: Adicionar campos conforme requisitos mudam
- **Integração de Dados**: Incorporar novas fontes de dados
- **Compliance**: Adicionar campos para regulamentações
- **Analytics**: Enriquecer dados para análises avançadas
- **Migration**: Migração gradual entre schemas

### Considerações Importantes:

- **Compatibilidade**: Nem todas as mudanças de tipo são suportadas
- **Performance**: Schemas muito largos podem impactar performance
- **Planejamento**: Evoluções devem ser planejadas considerando aplicações downstream
- **Testes**: Sempre testar evoluções em ambiente de desenvolvimento primeiro